In [83]:
import pandas as pd
import numpy as np

In [84]:
players = pd.read_csv('./raw_data/players_data.csv')
countries = pd.read_csv('./raw_data/country_rankings.csv')
clubs = pd.read_csv('./raw_data/club_tournament.csv')

In [85]:
players['market_value'] = players['market_value'].str.replace(' €', '')
players['market_value'] = players['market_value'].apply(lambda x: int(float(x[:-1]) * 1 if x[-1] == 'K' else float(x[:-1]) * 1000))
players['market_value'] = players['market_value'].astype(int)


players['age'] = players['age'].str.replace(' yrs', '')
players['age'] = players['age'].astype(int)
players['height'] = players['height'].str.replace(' cm', '')
players['height'] = players['height'].astype(int)

players = players.drop('birth_date', axis=1)

players = players[['name', 'market_value', 'age', 'height', 'preferred_foot', 'position', 'attacking', 'technical', 'tactical', 'defending', 'creativity', 'saves', 'anticipation', 'ball_distribution', 'aerial', 'avg_score_last_12_months', 'appearances_last_12_months', 'club', 'league', 'country']]

In [86]:
players.head()

,name,market_value,age,height,preferred_foot,position,attacking,technical,tactical,defending,creativity,saves,anticipation,ball_distribution,aerial,avg_score_last_12_months,appearances_last_12_months,club,league,country
0,Luciano Gondou,7400,22,189,Left,F,68,55,51,33,49,0,0,0,0,7.14,33,Argentinos Juniors,Liga Profesional de Fútbol,Argentina
1,Maximiliano Romero,2500,25,179,Right,F,62,48,54,32,47,0,0,0,0,6.86,37,Argentinos Juniors,Liga Profesional de Fútbol,Argentina
2,Gastón Verón,1200,22,184,Right,F,64,51,45,29,48,0,0,0,0,6.80,34,Argentinos Juniors,Liga Profesional de Fútbol,Argentina
3,Alan Lescano,2900,22,182,Left,M,58,60,47,38,59,0,0,0,0,6.88,37,Argentinos Juniors,Liga Profesional de Fútbol,Argentina
4,Leonardo Heredia,1200,28,187,Right,M,55,50,41,33,44,0,0,0,0,6.82,38,Argentinos Juniors,Liga Profesional de Fútbol,Argentina


In [87]:
players.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12421 entries, 0 to 12420
Data columns (total 20 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   name                        12421 non-null  object 
 1   market_value                12421 non-null  int32  
 2   age                         12421 non-null  int32  
 3   height                      12421 non-null  int32  
 4   preferred_foot              12421 non-null  object 
 5   position                    12421 non-null  object 
 6   attacking                   12421 non-null  int64  
 7   technical                   12421 non-null  int64  
 8   tactical                    12421 non-null  int64  
 9   defending                   12421 non-null  int64  
 10  creativity                  12421 non-null  int64  
 11  saves                       12421 non-null  int64  
 12  anticipation                12421 non-null  int64  
 13  ball_distribution           124

In [88]:
players.to_csv('./clean_data/players.csv', index=False)

In [89]:
countries = countries.drop(['slug', 'category_id', 'tournament_slug', 'tournament_id', 'tournament_season_id'], axis=1)

In [90]:
countries.head()

,rank,name,name_code,points,tournament_name
0,1,Argentina,ARG,1858.00,Liga Profesional de Fútbol
1,2,France,FRA,1840.59,Ligue 1
2,3,Belgium,BEL,1795.23,Pro League
3,4,England,ENG,1794.90,Premier League
4,5,Brazil,BRA,1788.65,Brasileirão Série A


In [91]:
countries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210 entries, 0 to 209
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   rank             210 non-null    int64  
 1   name             210 non-null    object 
 2   name_code        210 non-null    object 
 3   points           210 non-null    float64
 4   tournament_name  167 non-null    object 
dtypes: float64(1), int64(1), object(3)
memory usage: 8.3+ KB


In [92]:
countries.to_csv('./clean_data/countries.csv', index=False)

In [93]:
clubs = clubs.drop(['slug', 'id'], axis=1)
clubs = clubs[['name', 'short_name', 'name_code', 'position', 'tournament', 'country']]

In [94]:
clubs.head()

,name,short_name,name_code,position,tournament,country
0,Argentinos Juniors,Argentinos Jrs.,ARG,1,Liga Profesional de Fútbol,Argentina
1,Atlético Tucumán,Atlético Tucumán,CAT,2,Liga Profesional de Fútbol,Argentina
2,Boca Juniors,Boca Juniors,BOC,3,Liga Profesional de Fútbol,Argentina
3,Banfield,Banfield,BAN,4,Liga Profesional de Fútbol,Argentina
4,Barracas Central,Barracas,BAR,5,Liga Profesional de Fútbol,Argentina


In [95]:
clubs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2089 entries, 0 to 2088
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        2089 non-null   object
 1   short_name  2089 non-null   object
 2   name_code   2089 non-null   object
 3   position    2089 non-null   int64 
 4   tournament  2089 non-null   object
 5   country     2089 non-null   object
dtypes: int64(1), object(5)
memory usage: 98.1+ KB


In [96]:
clubs.to_csv('./clean_data/clubs.csv', index=False)